In [ ]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
cd MyDrive/MyDrive/Capstone/code_data

In [18]:
import pandas as pd
import numpy as np
from glob2 import glob
import json

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stpdfa
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from datetime import datetime
from datetime import date
from datetime import timedelta

from nltk import SnowballStemmer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package punkt to /Users/june/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/june/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [36]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)
# pd.set_option('display.max_rowwidth', 100)
pd.set_option('display.max_colwidth', 400)

# Functions

In [260]:
lmt = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
freqdist = nltk.FreqDist()
stop_words = set(stopwords.words('english'))
tweet_tokenizer = TweetTokenizer()

""" Replaces contractions from a string to their equivalents """
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not'), 
                         (r'i\'d', 'i would'), (r'I\'d', 'I would'), (r'he\'d', 'he would'), (r'she\'d', 'she would'), (r'they\'d', 'I would'), (r'we\'d', 'we would')]
def replaceContraction(text):
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

def capitalratio(tweet_text):
    uppers = [l for l in tweet_text if l.isupper()]
    capitalratio = len(uppers) / len(tweet_text)
    return capitalratio 

def getTokenization(sent):
    tweet_tokens = []
    sent = sent.lower()
    sent = replaceContraction(sent)

    sent = re.sub(r"http\S+", "*", sent) # http link -> '*'
    # sent = re.sub(r"@\S+", "@", sent)   # mention -> '@'
    sent = re.sub(r"@[^\s]+", "@", sent)   # mention -> '@'
    sent = re.sub(r"(#)(\S+)", r'\1 \2', sent) 

    sent = re.sub(r'([^\s\w@#\*]|_)+', '', sent) # Erasing Special Characters
    # sent = re.sub('@[^\s]+','atUser',sent)
    # sent = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',sent)
    # sent = re.sub(r'#([^\s]+)', r'\1', sent)


    # sent = re.sub('', '', sent.lower())
    # sent = [tweet_tokenizer.tokenize(sent)]
    sent = tweet_tokenizer.tokenize(sent)
    sent = [stemmer.stem(token) for token in sent]
    sent = [lmt.lemmatize(token) for token in sent]

    temp = [token for token in sent if not token in stop_words]
    url=0
    for token in temp:
        if token.startswith('*'):
            url+=1
    # tweet_tokens.append([temp])
    # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
    # df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])
    return temp, url

def extract_urls(entities_dicts):
    if len(entities_dicts) < 1:
        return 0
    if len(entities_dicts) == 1:
        return 1
    if len(entities_dicts) == 2:
        return 2

    # urls = []
    # urls_expanded = []

    # key = 'url'
    # key2 = 'expanded_url'
    # # print(len(entities_dict))
    # for i in entities_dicts:
    #     urls.append(i[key])
    #     urls_expanded.append(i[key2])
    # return 1, urls, urls_expanded

def getposcount(tokens):
    postag = []
    poscount = {}
    poscount['Noun']=0
    poscount['Verb']=0
    poscount['Adjective'] = 0
    poscount['Pronoun']=0
    poscount['FirstPersonPronoun']=0
    poscount['SecondPersonPronoun']=0
    poscount['ThirdPersonPronoun']=0
    poscount['Adverb']=0
    poscount['Numeral']=0
    poscount['Conjunction_inj']=0
    poscount['Particle']=0
    poscount['Determiner']=0
    poscount['Modal']=0
    poscount['Whs']=0
    poscount['HashTag'] = 0
    Nouns = {'NN','NNS','NNP','NNPS'}
    Adverbs = {'RB','RBR','RBS'}
    Whs = {'WDT','WP','WRB'} # Composition of wh-determiner(that,what), wh-pronoun(who), wh-adverb(how)
    Verbs={'VB','VBP','VBZ','VBN','VBG','VBD','To'}
    first_person_pronouns=['i','I','me','my','mine','we','us','our','ours'] #'i',
    second_person_pronouns=['you','your','yours']
    third_person_pronouns=['he','she','it','him','her','it','his','hers','its','they','them','their','theirs']

    for word in tokens:
        w_lower=word.lower()
        if w_lower in first_person_pronouns:
            poscount['FirstPersonPronoun']+=1
        elif w_lower in second_person_pronouns:
            poscount['SecondPersonPronoun']+=1
        elif w_lower in third_person_pronouns:
            poscount['ThirdPersonPronoun']+=1
    
    postag = nltk.pos_tag(tokens)
    for g1 in postag:
        if g1[1] == '#':
            poscount['HashTag'] += 1
        elif g1[1] in Nouns:
            poscount['Noun'] += 1
        elif g1[1] in Verbs:
            poscount['Verb']+= 1
        elif g1[1]=='ADJ'or g1[1]=='JJ':
            poscount['Adjective']+=1
        elif g1[1]=='PRP' or g1[1]=='PRON' or g1[1]=='PRP$':
            poscount['Pronoun']+=1
        elif g1[1] in Adverbs or g1[1]=='ADV':
            poscount['Adverb']+=1
        elif g1[1]=='CD':
            poscount['Numeral']+=1
        elif g1[1]=='CC' or g1[1]=='IN':
            poscount['Conjunction_inj']+=1
        elif g1[1]=='RP':
            poscount['Particle']+=1
        elif g1[1]=='MD':
            poscount['Modal']+=1
        elif g1[1]=='DT':
            poscount['Determiner']+=1
        elif g1[1] in Whs:
            poscount['Whs']+=1
    return poscount

def fetchRawText(path, events, tweetType):
    jsons = []
    for i, event in enumerate(events):
        jsons.append(glob('%s/%s/**/%s/*.json' % (path, event,tweetType)))
    for i,d in enumerate(jsons): print("%s's length is %d" %(events[i], len(d)))

    targets = []
    features = []
    for index, dataset in enumerate(jsons):
        targetEvent = []
        dataEvent = []
        count = 0  # help var
        for jsonFile in dataset:
            count += 1
            if jsonFile.find("non-rumours") == -1:
                targetEvent.append(1)
            else:
                targetEvent.append(0)

            with open(jsonFile, 'r') as f:
                for l in f.readlines():
                    if not l.strip():  # skip empty lines
                        continue
                    json_data = json.loads(l)
                    # print (json_data,"\n\n")
                    dataEvent.append(json_data)
        print(index, events[index], len(targetEvent), len(dataEvent))
        targets.append(targetEvent)
        features.append(dataEvent)

    # print("\nNumber of Events:", len(targets))
    # print("Number of tweets in the first event:", len(targets[0]))

    # targets은 targetEvent들을 리스트에 담은 것
    target_list = []
    for event in targets:
        for elem in event:
            target_list.append(elem)
    target = pd.DataFrame(target_list, columns=["target"])

    extracted_features = []

    extracted = []

    for obj_list in features:
        extracted_event = []
        for obj in obj_list:
            output_f = dict()
            output_f['text'] = obj['text']
            urls_dicts = obj['entities']['urls']
            output_f['URLcount'] = extract_urls(urls_dicts)
        
            # print(type(obj['user']))
            # print(obj['user'].contains_key('entities'))
            # if ('url' in obj['user']):
            #     output_f['hasUserURL'] = 1
            #     output_f['user_url'] = 1 if (obj['user']['url'] != None) else 0
            # elif ('entities' in obj['user']):
            #     output_f['user_entity'] = obj['user']['entities']['url']['urls']
            #     # print(obj['user']['entities']['url']['urls'])
            #     output_f['user_url'] = obj['user']['entities']['expanded_url']
            #     output_f['hasUserURL'] , _ , output_f['user_url'] = extract_urls(obj['user']['entities']['url']['urls'])
            # else:
            #     # output_f['user_entity'] = None
            #     output_f['user_url'] = 0
            #     output_f['hasUserURL'] = 0
            

            output_f['text_token'], output_f['URLcount'] = getTokenization(obj['text'])

            '''POS Tagging'''
            temp = output_f['text']
            temp = re.sub(r"(#)(\S+)", r'\1 \2', temp)
            temp = re.sub(r"http\S+", "", temp)
            temp = replaceContraction(temp.lower())
            temp = temp.split()
            pos_dict=getposcount(temp)
            output_f.update(pos_dict)

            output_f['char_count'] = len(output_f['text'])
            output_f['word_count'] = len(output_f['text'].split())

            output_f['has_question'] = "?" in output_f["text"]
            output_f['has_exclaim'] = "!" in output_f["text"]
            output_f['has_period'] = "." in output_f["text"]

            output_f['capital_ratio']=(capitalratio(obj['text']))
            output_f['tweet_count'] = np.log2(obj['user']['statuses_count'])
            output_f['listed_count'] = obj['user']['listed_count']
            output_f['follow_ratio'] = np.log2(obj['user']['followers_count'])

            acc_created = datetime.strptime(obj['user']['created_at'], '%a %b %d %H:%M:%S %z %Y')
            tweet_created = datetime.strptime(obj['created_at'], '%a %b %d %H:%M:%S %z %Y')
            age = (tweet_created - acc_created)
            output_f['capital_ratio']=(capitalratio(obj['text']))
            output_f['verified'] = obj['user']['verified']

            extracted_event.append(output_f)
        extracted_features.append(extracted_event)

    extracted_df = []
    for i, data in enumerate(extracted_features):
        temp = pd.DataFrame(data)
        temp["Event"] = events[i]
        extracted_df.append(pd.DataFrame(temp))

    final = pd.concat(extracted_df, ignore_index=True)
    final = pd.concat([final, target], axis=1)
    return final
    

# PHEME

In [261]:
path = "../pheme-rnr-dataset"
events = ['charliehebdo', 'ferguson',
          'germanwings-crash', 'ottawashooting', 'sydneysiege']
# events = ['ferguson']
tweetType = 'source-tweet'
jsons = []
final = fetchRawText(path, events, tweetType)
target = final.target
final.drop(['text_token','text','Event','target'], axis=1, inplace=True)
final.head()

charliehebdo's length is 2079
ferguson's length is 1143
germanwings-crash's length is 469
ottawashooting's length is 890
sydneysiege's length is 1221
0 charliehebdo 2079 2079
1 ferguson 1143 1143
2 germanwings-crash 469 469
3 ottawashooting 890 890
4 sydneysiege 1221 1221


,URLcount,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,Numeral,Conjunction_inj,Particle,Determiner,Modal,Whs,HashTag,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,verified
0,1,7,2,0,0,0,0,0,0,0,2,0,0,0,0,0,88,12,False,False,True,0.159091,15.956172,7177,17.564194,True
1,0,3,2,0,0,0,0,0,0,0,1,0,0,0,0,1,53,6,False,False,True,0.037736,10.071462,140,12.201205,False
2,0,4,4,7,0,0,0,0,1,0,2,0,0,0,0,2,136,18,False,False,True,0.073529,12.810170,758,14.316352,False
3,2,4,5,1,0,0,0,0,0,0,0,0,2,0,1,1,138,16,False,False,True,0.101449,15.732035,102287,23.876904,True
4,2,7,2,0,0,0,0,0,0,0,2,0,0,0,0,1,117,13,False,False,True,0.145299,16.680002,13583,19.683865,True


# PHEME (ADDITIONAL)

In [239]:
path = "../PHEME/all-rnr-annotated-threads"
events = ['ebola-essien', 'prince-toronto', 'putinmissing']
tweetType = 'source-tweets'
jsons = []
final_ext = fetchRawText(path,events,tweetType)
ext_target = final_ext.target
final_ext.drop(['text_token','text','Event','target'], axis=1, inplace=True)
final_ext.head()


ebola-essien's length is 14
prince-toronto's length is 233
putinmissing's length is 238
0 ebola-essien 14 14
1 prince-toronto 233 233
2 putinmissing 238 238


<ipython-input-214-5f0b58142d99>:224: RuntimeWarning: divide by zero encountered in log2
  output_f['listed_count'] = np.log2(obj['user']['listed_count'])


,URLcount,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,Numeral,Conjunction_inj,Particle,Determiner,Modal,Whs,HashTag,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,verified
0,1,2,2,1,0,0,0,0,0,0,1,0,1,0,0,0,69,8,False,False,True,0.101449,15.311890,7.209453,14.414223,False
1,0,9,6,3,0,2,0,0,3,0,4,0,1,1,0,0,148,25,False,False,True,0.027027,8.991522,10.664447,18.897987,True
2,0,7,4,1,1,0,0,1,0,0,2,0,3,0,1,0,119,20,False,False,True,0.025210,16.344851,11.080151,17.825921,False
3,2,5,3,2,0,0,0,0,0,0,1,0,2,0,0,0,130,16,False,False,True,0.107692,13.915132,9.247928,16.166398,True
4,1,4,4,2,0,0,0,0,1,0,2,0,1,0,0,0,120,15,False,False,True,0.066667,16.344851,11.080151,17.825921,False


# CLF

In [243]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from sklearn.naive_bayes import GaussianNB

In [244]:
def train_test(X_train, X_test, y_train, y_test, clf):
    clf.fit(X_train, y_train)
    result = clf.predict(X_test)
    print("Accuracy:",accuracy_score(result,y_test))
    print(classification_report(y_test, result))
    print(f1_score(y_test, result, average='macro'))
    

def valid(X_valid, y_valid, clf):
    clf.fit
    result = clf.predict(X_valid)
    print("Accuracy:",accuracy_score(result,y_valid))
    print(classification_report(y_valid, result))

def cv_events(data):
    NUM_EVENT = data.Event.unique().shape[0]
    EVENTS = data.Event.unique()

    cv_pd_list = []
    for i, d in enumerate(EVENTS):
        df1, df2 = [x for _, x in data.groupby(data['Event'] != d)]
        df1.reset_index(inplace=True, drop=True)
        df2.reset_index(inplace=True, drop=True)
        cv_pd_list.append([df1, df2])
    return cv_pd_list


In [252]:
final = final.fillna(0)
final_ext = final_ext.fillna(0)

In [259]:
final_ext.listed_count.value_counts

,URLcount,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,Numeral,Conjunction_inj,Particle,Determiner,Modal,Whs,HashTag,char_count,word_count,capital_ratio,tweet_count,listed_count,follow_ratio
count,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000
mean,0.882799,5.649259,2.744226,1.787832,0.452947,0.214926,0.076181,0.211824,0.579283,0.303171,2.113409,0.061875,1.011203,0.110651,0.178904,1.059807,118.605998,16.205963,0.092076,14.539746,-inf,16.293679
std,0.764258,2.004272,1.717028,1.281310,0.800325,0.540545,0.348446,0.527909,0.831424,0.653016,1.330129,0.250766,1.110067,0.345124,0.427526,0.913822,23.175330,4.572858,0.070563,2.211958,NaN,3.980507
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,3.000000,0.000000,2.000000,-inf,3.807355
25%,0.000000,4.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,107.000000,13.000000,0.052106,13.328079,7.734710,13.372593
50%,1.000000,6.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,1.000000,127.000000,16.000000,0.081081,14.786321,10.228819,16.223983
75%,1.000000,7.000000,4.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000,2.000000,0.000000,0.000000,1.000000,137.000000,19.000000,0.115108,16.365596,13.053714,19.488984
max,8.000000,15.000000,10.000000,8.000000,6.000000,5.000000,4.000000,5.000000,6.000000,10.000000,8.000000,3.000000,9.000000,3.000000,4.000000,8.000000,152.000000,31.000000,0.909091,18.988324,21.117830,24.592810


In [258]:
final_ext.has_question

0      False
1      False
2      False
3      False
4      False
       ...  
480    False
481    False
482    False
483    False
484    False
Name: has_question, Length: 485, dtype: bool

In [257]:
clf = GaussianNB()
train_test(final.has_question, final_ext.has_question, target, ext_target, clf)

ValueError: Expected 2D array, got 1D array instead:
array=[False False False ... False False False].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
final_ext.listed_count.value_counts